# Titolo

...

In [208]:
import numpy as np
import pandas as pd
import re
import os
import random
import pprint
from collections import defaultdict

In [209]:
def remove_nan(df:pd.DataFrame) -> dict:
    
    """
    Rimuove i valori nulli da una lista
    """

    lookup_dict = df.to_dict('list')

    for k, v in lookup_dict.items():

        while np.nan in lookup_dict[k]:
            lookup_dict[k].remove(np.nan)

    return lookup_dict

## Parte 1

### Costruzione delle tabelle di lookup

Servono per generare nuovi dati ricombinando elementi già noti

In [210]:
df_entities = pd.read_excel('dataset.xlsx',sheet_name='entities_slots')

lookups = remove_nan(df_entities)

In [211]:
# pprint.pprint(lookups)

### Estrazione frasi utente

In [212]:
df_user = pd.read_excel('dataset.xlsx',sheet_name='user', header=None)
df_user.columns = ['user','sentences']
df_user['user'] = df_user['user'].fillna(method='ffill', axis=0)

sentences = defaultdict(list)

df_grouped = df_user.groupby('user')

for group in df_grouped.groups:
    
    sentences[group] = df_grouped.get_group(group)['sentences'].tolist()

In [7]:
# pprint.pprint(sentences)

### Generazione del file txt contenente le frasi

In [213]:
sentences_file = "sentences_origin.txt"

In [214]:
i = True

for k, v in sentences.items():
    
    for values in v:
        
        if i:
            with open(sentences_file, 'w+') as f:
                f.writelines(f'{values}|{k}\n')
                i = False
        else:
            with open(sentences_file, 'a+') as f:
                f.writelines(f'{values}|{k}\n')

### Estrazione frasi bot

In [215]:
# df_bot = pd.read_excel('dataset.xlsx',sheet_name='bot')
# df_bot.head()


df_bot = pd.read_excel('dataset.xlsx',sheet_name='bot', header=None)
df_bot.columns = ['bot','sentences']
df_bot['bot'] = df_bot['bot'].fillna(method='ffill', axis=0)

bot_sentences = defaultdict(list)

df_grouped = df_bot.groupby('bot')

for group in df_grouped.groups:
    
    bot_sentences[group] = df_grouped.get_group(group)['sentences'].tolist()


In [216]:
# bot_sentences = remove_nan(df_bot)
bot_sentences

defaultdict(list,
            {'bot_ReplyCity ': ['Nella città di [SEDE] ci sono %d fattorie didattiche'],
             'bot_ReplyProv': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche'],
             'bot_ReplyProvWithParams': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche con le caratteristiche richieste']})

### Estrazione frasi dialogo

In [217]:
df_dialogs = pd.read_excel('dataset.xlsx',sheet_name='dialogs')
df_dialogs.head()

,Dialog_1,Dialog_2,Dialog_3
0,user_AskProv,user_AskCity,user_AskProvWithParams
1,bot_ReplyProv,bot_ReplyCity,bot_ReplyProvWithParams


In [218]:
dialogs = remove_nan(df_dialogs) 
dialogs

{'Dialog_1': ['user_AskProv', 'bot_ReplyProv'],
 'Dialog_2': ['user_AskCity', 'bot_ReplyCity '],
 'Dialog_3': ['user_AskProvWithParams', 'bot_ReplyProvWithParams']}

### Generazione di nuove frasi ricombinandole con le lookup tables

Si estraggono le frasi e le categorie di appartenenza per archiviarle in due liste separate

In [219]:
sentences, categories = [], []

with open(sentences_file, encoding='utf-8') as f:
    dataset = f.read()
    dataset = dataset.split("\n")

for data in dataset:
    sentence = data.split("|")
    
    if len(sentence) > 1:
        
        # TODO: Lasciare upper ?
        
        sentences.append(sentence[0].upper())
        categories.append(sentence[1])
        
assert len(sentences) == len(categories)

In [220]:
sentences[0]

'CI SONO FATTORIE DIDATTCHE AD [SEDE](EBOLI)?'

In [221]:
sentences_file_generated = "sentences_generated.txt"

In [222]:
"""
Numero di frasi che verranno casualmente generate
"""

n_sentences = 1000

In [223]:
slots = list(lookups.keys())

for i in range(n_sentences):
        
    index = random.randint(0, len(sentences)-1)

    sentence = sentences[index]
    category = categories[index]
    
    for key in slots:
        
        """
        Ogni volta che regex individua lo slot nella frase
            sostituisce il valore con uno estratto in modo casuale
        """
        
        # TODO: Verificare

        regex_str = fr'\[{key}]\((?P<value>[a-z ]+)\)+'
        
        slot_match = re.compile(regex_str)

        repl = f"[{key}]({random.choice(lookups[key])})"

        sentence = slot_match.sub(repl, sentence)
        
        
        """
        Poi riassocia la categoria di partenza
        """
        
    if i == 0:
        
        with open(sentences_file_generated, 'w+') as f:
            f.writelines(f'{sentence}|{category}\n')
        
    else:
    
        with open(sentences_file_generated, 'a+') as f:
            f.writelines(f'{sentence}|{category}\n')

## Parte 2

...

...

In [224]:
def rimuovi_punteggiatura(text):
    
    text = re.sub(r'[\.,;:!?]' , " ", text)
    text = re.sub(r'\s+' , " ", text)
    
    return text

In [225]:
with open(sentences_file_generated, encoding='utf-8') as f:
    sentences = f.read()
    sentences = sentences.split("\n")

In [226]:
example = random.choice(sentences)
example, _ = example.split("|")
print("Before:",example)

example = re.sub(r'\[(?P<name>[a-zA-Z_]+)\]|\(|\)+', "", example)
print("After:",example)

Before: CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI [PROVINCIA](NAPOLI)?
After: CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI NAPOLI?


### Inizio preparazione dati per l'addestramento dell'algoritmo CRF

In [227]:
sentence = sentences[10]
print(f"Fase iniziale: {sentence}")

sentence, categ = sentence.split("|")
sentence = rimuovi_punteggiatura(sentence)

Fase iniziale: QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI [PROVINCIA](SALERNO)?|user_AskProv


In [228]:
regex_str = r'\[(?P<name>[a-zA-Z_]+)\]\((?P<value>[a-zA-Z\' ]+)\)+'
slot_match = re.compile(regex_str)

##### Con il metodo split possiamo confrontare se ogni elemento della lista appartiene allo slot, al valore o a nessuno dei due.

In [229]:
splits = slot_match.split(sentence)
splits

['QUANTE FATTORIE DIDATTICHE CI SONO IN PROVINCIA DI ',
 'PROVINCIA',
 'SALERNO',
 ' ']

In [230]:
matches = slot_match.findall(sentence)

dct = {k:v for k,v in matches}
dct

{'PROVINCIA': 'SALERNO'}

##### L'obiettivo finale è quello di trasportare i dati in colonne dove  ad ogni riga corrisponde una parola e ogni parola può appartenere o meno ad uno slot

In [231]:
for split in splits:
   
    if split in list(dct.values()):
        for value in split.split():
            
            index = list(dct.values()).index(split)
            key = list(dct.keys())[index]
            
            print(value, "->", key)
    elif split == dct.keys():
        pass
    else:
        for splt in split.split():
            print(splt, "->", '0')

QUANTE -> 0
FATTORIE -> 0
DIDATTICHE -> 0
CI -> 0
SONO -> 0
IN -> 0
PROVINCIA -> 0
DI -> 0
PROVINCIA -> 0
SALERNO -> PROVINCIA


#### Procediamo ad applicare la trasformazione a tutto il dataset

In [232]:
arr_sentences = list()
arr_categories = list()

for n, sentence in enumerate(sentences):

    try:
    
        sentence, categ = sentence.split("|")
        
        sentence = rimuovi_punteggiatura(sentence)

        arr_categories.append(categ)

        splits = slot_match.split(sentence)

#         match = slot_match.search(frase)
        matches = slot_match.findall(sentence)
    
        dct = {k:v for k,v in matches}        

        if matches is not None:
            
            for split in splits:
                if split in list(dct.values()):
                    for value in split.split():
                        
                        index = list(dct.values()).index(split)
                        key = list(dct.keys())[index]
                        
                        arr_sentences.append([n, value, key])
                elif split in list(dct.keys()):
                    pass
                else:
                    for value in split.split():
                        arr_sentences.append([n, value, 'O'])
                        
        else:
            """
            Serve per verificare se in qualche frase non avviene il match
            """
            print(n, frase)
            
    except Exception as err:
        pass


arr_sentences[:10]

[[0, 'QUANTE', 'O'],
 [0, 'FATTORIE', 'O'],
 [0, 'DIDATTICHE', 'O'],
 [0, 'CI', 'O'],
 [0, 'SONO', 'O'],
 [0, 'IN', 'O'],
 [0, 'PROVINCIA', 'O'],
 [0, 'DI', 'O'],
 [0, 'SALERNO', 'PROVINCIA'],
 [0, 'CON', 'O']]

In [233]:
df = pd.DataFrame(arr_sentences, columns=['n_frase','word','tag'])
df.head(10)

,n_frase,word,tag
0,0,QUANTE,O
1,0,FATTORIE,O
2,0,DIDATTICHE,O
3,0,CI,O
4,0,SONO,O
5,0,IN,O
6,0,PROVINCIA,O
7,0,DI,O
8,0,SALERNO,PROVINCIA
9,0,CON,O


#### Preparazione della variabile target ( y per renderla più familiare... )

In [234]:
df_target = df[['n_frase','tag']]
df_target.head(10)

,n_frase,tag
0,0,O
1,0,O
2,0,O
3,0,O
4,0,O
5,0,O
6,0,O
7,0,O
8,0,PROVINCIA
9,0,O


In [235]:
y = list()

for k, v in df_target.groupby('n_frase'):
        
    y.append(v['tag'].tolist())

In [236]:
y[:5]

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'O', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA']]

##### Adesso trasformiamo tutto in una funzione

In [237]:
def prepare_target_crf(df:pd.DataFrame) -> list:
    
    """
    Ultimo step per preparare i target per addestrare l'algoritmo
    
    Parameters:
    -----------
    
    df : pd.DataFrame
    
        il DataFrame deve contenere due colonne, una che indicizza la frase
        e l'altra che indica se il valore è uno slot o meno
    
    
    Returns:
    -----------
    y : list
    
        una lista annidata
    
    """
    
    y = list()

    for k, v in df.groupby('n_frase'):

        y.append(v['tag'].tolist())
    
    return y

In [238]:
y = prepare_target_crf(df_target)
y[:5]

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'O', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA', 'O', 'CATEGORIA'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'PROVINCIA']]

#### Data Augmentation

...spiegare meglio il motivo...

In [240]:
df['shift-3'] = df.groupby('n_frase')['word'].shift(1).str.slice(-3)
df['shift+3'] = df.groupby('n_frase')['word'].shift(-1).str.slice(0,3)
df['shift-3'].fillna('BOF', inplace=True)
df['shift+3'].fillna('EOF', inplace=True)

df['bias'] = 1

df.head(10)

,n_frase,word,tag,shift-3,shift+3,bias
0,0,QUANTE,O,BOF,FAT,1
1,0,FATTORIE,O,NTE,DID,1
2,0,DIDATTICHE,O,RIE,CI,1
3,0,CI,O,CHE,SON,1
4,0,SONO,O,CI,IN,1
5,0,IN,O,ONO,PRO,1
6,0,PROVINCIA,O,IN,DI,1
7,0,DI,O,CIA,SAL,1
8,0,SALERNO,PROVINCIA,DI,CON,1
9,0,CON,O,RNO,ALL,1


In [251]:
# for k, frase in df.groupby(by='n_frase')['word']:
#     print(" ".join(frase))
#     print(frase)

In [33]:
def extend_data(df:pd.DataFrame) -> pd.DataFrame:
    
    """
    Estende i dati attraverso un algortimo personalizzato
    """
    
    df['shift-3'] = df.groupby('n_frase')['word'].shift(1).str.slice(-3)
    df['shift+3'] = df.groupby('n_frase')['word'].shift(-1).str.slice(0,3)
    df['shift-10'] = df.groupby('n_frase')['word'].shift(1).str.slice(-10)
    df['shift+10'] = df.groupby('n_frase')['word'].shift(-1).str.slice(0,10)

    df['shift-3'].fillna('BOF', inplace=True)
    df['shift+3'].fillna('EOF', inplace=True)
    df['shift-10'].fillna('BOF', inplace=True)
    df['shift+10'].fillna('EOF', inplace=True)


    df['bias'] = 1
    
    try:
        df.drop(columns=['tag'], inplace=True)
    except:
        pass

    
    return df

In [34]:
df = extend_data(df)
df.head(10)

,n_frase,word,shift-3,shift+3,bias,shift-10,shift+10
0,0,CI,BOF,SON,1,BOF,SONO
1,0,SONO,CI,FAT,1,CI,FATTORIE
2,0,FATTORIE,ONO,DID,1,SONO,DIDATTCHE
3,0,DIDATTCHE,RIE,AD,1,FATTORIE,AD
4,0,AD,CHE,EBO,1,DIDATTCHE,EBOLI
5,0,EBOLI,AD,EOF,1,AD,EOF
6,1,CI,BOF,SON,1,BOF,SONO
7,1,SONO,CI,FAT,1,CI,FATTORIE
8,1,FATTORIE,ONO,DID,1,SONO,DIDATTICHE
9,1,DIDATTICHE,RIE,IN,1,FATTORIE,IN


#### Preparazione dei varibili di training ( X per renderli più familiari... )

In [35]:
X = list()

for k, v in df.groupby('n_frase'):
    
    v.drop(columns='n_frase', inplace=True)

    X.append(v.to_dict('records'))

In [36]:
X[0]

[{'word': 'CI',
  'shift-3': 'BOF',
  'shift+3': 'SON',
  'bias': 1,
  'shift-10': 'BOF',
  'shift+10': 'SONO'},
 {'word': 'SONO',
  'shift-3': 'CI',
  'shift+3': 'FAT',
  'bias': 1,
  'shift-10': 'CI',
  'shift+10': 'FATTORIE'},
 {'word': 'FATTORIE',
  'shift-3': 'ONO',
  'shift+3': 'DID',
  'bias': 1,
  'shift-10': 'SONO',
  'shift+10': 'DIDATTCHE'},
 {'word': 'DIDATTCHE',
  'shift-3': 'RIE',
  'shift+3': 'AD',
  'bias': 1,
  'shift-10': 'FATTORIE',
  'shift+10': 'AD'},
 {'word': 'AD',
  'shift-3': 'CHE',
  'shift+3': 'EBO',
  'bias': 1,
  'shift-10': 'DIDATTCHE',
  'shift+10': 'EBOLI'},
 {'word': 'EBOLI',
  'shift-3': 'AD',
  'shift+3': 'EOF',
  'bias': 1,
  'shift-10': 'AD',
  'shift+10': 'EOF'}]

##### Adesso trasformiamo tutto in una funzione

In [37]:
def prepare_data_crf(df:pd.DataFrame) -> list:
    
    """
    Ultimo step per preparare i dati per addestrare l'algoritmo
    
    Parameters:
    -----------
    
    df : pd.DataFrame
    
        il numero di colonne del DataFrame dipende da come è stata impostata
        la Data Augmentation... l'importante è che il df contenga
        la colonna 'n_frase' usata come indice
    
    
    Returns:
    -----------
    y : list
    
        una lista annidata di dictionary
    
    """

    
    X = list()

    for k, v in df.groupby('n_frase'):

        v.drop(columns='n_frase', inplace=True)

        X.append(v.to_dict('records'))
    
    return X

In [38]:
X = prepare_data_crf(df)

#### L'algoritmo di Conditional Random Field

In [39]:
import sklearn_crfsuite

In [40]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_states=False,     # Default
    all_possible_transitions=False # Default
)

In [41]:
crf.fit(X, y)

/home/ubuntupc/.local/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_states=False,
    all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,
    calibration_candidates=None, calibration_eta=None,
    calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

#### Un esempio su una frase del trainset

In [42]:
sentences[5].split("|")[0]

'CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [PROVINCIA](SALERNO)?'

In [43]:
for n, word in enumerate(X[5]):
    
    print(f"{n} --> {word['word']}")

0 --> CI
1 --> SONO
2 --> FATTORIE
3 --> DIDATTICHE
4 --> IN
5 --> PROVINCIA
6 --> DI
7 --> SALERNO


In [44]:
y_pred = crf.predict_single(X[5])

for n, pred in enumerate(y_pred):
    
    print(f"{n} --> {pred}")

0 --> O
1 --> O
2 --> O
3 --> O
4 --> O
5 --> O
6 --> O
7 --> PROVINCIA


#### Test su una nuova frase

In [45]:
nuova_frase = 'CI SONO FATTORIE DIDATTICHE, AD AFRAGOLA'

In [46]:
def prepare_sentence(sentence:str) -> [pd.DataFrame, list]:
    
    """
    Prepare la frase per il predict
    
    Parameters:
    -----------
    
    sentence : str
    
        è la frase che sarà elaborata
    
    
    Returns:
    -----------
    df : DataFrame
    
    X_arr[0] : array
        
        dati in formato utile a CRF per il predict
    
    """

    sentence = rimuovi_punteggiatura(sentence)
    
    X_arr = list()
    
    df = pd.DataFrame(data = [i for i in sentence.split()], columns=['word'])
    df['n_frase'] = 1
    
    df = extend_data(df)
    
    
    for k, v in df.groupby('n_frase'):

        v.drop(columns='n_frase', inplace=True)

        X_arr.append(v.to_dict('records'))
    
    
    return df, X_arr[0]

In [47]:
new_df = pd.DataFrame()

new_df = pd.DataFrame({'n_frase':1,
                       'word':[i for i in nuova_frase.split()]})

In [48]:
df, X_arr = prepare_sentence(nuova_frase)

crf.predict_single(X_arr)

['O', 'O', 'O', 'O', 'O', 'SEDE']

##### L'algoritmo ha classificato bene... costruiamo adesso lo "Slots extractor"

In [49]:
def extend_sentence(sentence:str, model:sklearn_crfsuite.estimator.CRF) -> pd.DataFrame:
    
   
    """
    Estrae slots e lo aggiunge al DataFrame come colonna
    
    Parameters:
    -----------
    
    sentence : str
    
        è la stringa che contiene la frase
        
    model : sklearn_crfsuite.estimator.CRF
        
        è il modello addestrato di ConditionalRandomField
    
    
    Returns:
    -----------
    df : DataFrame
    
        al DataFrame di partenza viene aggiunta una colonna 
        con l'indicazione del tipo di slot individuato
       
    """
    
    
    df, X_arr = prepare_sentence(sentence)
    
    df['slots'] = model.predict_single(X_arr)
    
    return df

In [50]:
extend_sentence(nuova_frase, crf)

,word,n_frase,shift-3,shift+3,shift-10,shift+10,bias,slots
0,CI,1,BOF,SON,BOF,SONO,1,O
1,SONO,1,CI,FAT,CI,FATTORIE,1,O
2,FATTORIE,1,ONO,DID,SONO,DIDATTICHE,1,O
3,DIDATTICHE,1,RIE,AD,FATTORIE,AD,1,O
4,AD,1,CHE,AFR,DIDATTICHE,AFRAGOLA,1,O
5,AFRAGOLA,1,AD,EOF,AD,EOF,1,SEDE


In [58]:
import requests
import json
import datetime
import dateparser

def extract_datetime(text:str, url = "http://0.0.0.0:8000/parse"):
    
    data = {"locale":"it_IT",
        "text":text}
    
#     resp = dict()
#     resp['datetime'] = list()

    resp = None
    
    datetimes = list()
    
    try:
    
        response = requests.post(url, data=data)
        
        try:
        
            if response.status_code == 200:

                for dt in response.json():
                    
                    if dt['dim'] == "time":

                        dtime = dt['value']['value']
                        dtime = dateparser.parse(dtime)

                        datetimes.append(dtime)

            resp = dict()
            
            if len(datetimes) > 1:
                resp['datetime'] = list([min(datetimes), max(datetimes)])
            else:
                resp['datetime'] = list(datetimes)
            
        except:
            pass
        
    except:
        pass
    
    
    return resp

In [180]:
def slots_extractor(sentence:str, model:sklearn_crfsuite.estimator.CRF) -> defaultdict:
    
    
    """
    Restituisce un dictionary degli slots individuati
    
    Parameters:
    -----------
    
    sentence : str
    
        è la stringa che contiene la frase
        
    model : sklearn_crfsuite.estimator.CRF
        
        è il modello addestrato di ConditionalRandomField
    
    
    Returns:
    -----------
    dd : dictionary
           
    """
    
    df = extend_sentence(sentence, model)
    
    dd = defaultdict(list)

    for k, v in df.query("slots != 'O'").groupby('slots'):
        dd[k] = " ".join(v['word'])
        
    
    
    """
    Estrazione date ed orari tramite duckling    
    """
    
    date_time = extract_datetime(sentence)

    if date_time is not None and len(date_time) > 0:
        dd['DATETIMES'] = date_time['datetime']
#     else:
#         sentence_dict['datetimes'] = False


    return dd

In [199]:
slots_extractor('CI SONO FATTORIE DIDATTICHE, AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 ?', crf)

defaultdict(list,
            {'CATEGORIA': '8',
             'DATETIMES': [datetime.datetime(2021, 9, 5, 8, 0, tzinfo=<StaticTzInfo 'UTC\-07:00'>)]})

## Parte 3

### La costruzione dell'algoritmo del la classificazione degli intents

L'addestramento deve avvenire con le frasi "ripulite" dai dati per addestrare il ConditionalRandomField... ovvero useremo solo gli slots per diminuire lo spazio dimensionale

In [53]:
df = pd.read_csv(sentences_file_generated,sep="|", header=None)

df.columns = ['sentences', 'intents']
df.head()

,sentences,intents
0,CI SONO FATTORIE DIDATTCHE AD [SEDE](EBOLI)?,user_AskCity
1,CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [P...,user_AskProvWithParams
2,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv
3,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv
4,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv


In [182]:
def conserva_solo_slot_name(text:str) -> str:
    
    """
    Per facilitare la riduzione dello spazio dimensionale 
        vengono eliminati i valori degli slots
        mentre vengono conservati i loro nomi
        
        
    Parameters:
    -----------
    
    text : str
    
        è la stringa che contiene la frase
        
    
    Returns:
    -----------
    text : str
    
    """
        
    text = rimuovi_punteggiatura(text)
    
    pattern = r'(\([A-Za-z0-9 ]+\)|\[|\])'
    
    text = re.sub(pattern, "", text)
    
    return text

In [55]:
df['sentences'][5]

'CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI [PROVINCIA](SALERNO)?'

In [56]:
conserva_solo_slot_name(df['sentences'][100])

'CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI PROVINCIA '

In [57]:
df['sentences'] = df['sentences'].apply(conserva_solo_slot_name)

df.head()

,sentences,intents
0,CI SONO FATTORIE DIDATTCHE AD SEDE,user_AskCity
1,CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI PR...,user_AskProvWithParams
2,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv
3,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv
4,CI SONO FATTORIE DIDATTICHE NELLA PROVINCIA DI...,user_AskProv


#### Funzioni per addestrare l'algoritmo di classificazione

...le due funzioni che seguono si spiegano meglio con un esempio...

In [200]:
def replace_slot_values(sentence_dict:dict) -> dict:
    
    """
    Integrazione dictionary - parte 1 di 2
    
    
    Per operare una riduzione delle variabili
    sostituisce il valore con il relativo slot
    """
    
    sentence_dict['replaced_sentence'] =  sentence_dict['sentence']
    
    for k, v in sentence_dict['slots'].items():
        
        if k != 'DATETIMES':
        
            sentence_dict['replaced_sentence'] = re.sub(v, k, sentence_dict['replaced_sentence'])
    
    
    return sentence_dict

In [201]:
def add_slots(sentence:str, model:sklearn_crfsuite.estimator.CRF) -> dict:
    
    """
    Integrazione dictionary - parte 2 di 2
    """
    
    sentence_dict = {}
    
    sentence = rimuovi_punteggiatura(sentence)
    
    slots_dict = slots_extractor(sentence, model)
    
    sentence_dict['sentence'] = sentence
    sentence_dict['slots'] = slots_dict
    
    sentence_dict = replace_slot_values(sentence_dict)
    
#     date_time = extract_datetime(sentence)
    
#     if date_time is not None and len(date_time) > 0:
#         sentence_dict['datetimes'] = date_time['datetime']
#     else:
#         sentence_dict['datetimes'] = False
    
    return sentence_dict

In [202]:
nuova_frase

'CI SONO FATTORIE DIDATTICHE, AD AFRAGOLA'

In [204]:
add_slots('CI SONO FATTORIE DIDATTICHE, AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 DI SERA ?', crf)

{'sentence': 'CI SONO FATTORIE DIDATTICHE AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 DI SERA ',
 'slots': defaultdict(list,
             {'PROVINCIA': 'SERA',
              'DATETIMES': [datetime.datetime(2021, 9, 5, 20, 0, tzinfo=<StaticTzInfo 'UTC\-07:00'>)]}),
 'replaced_sentence': 'CI SONO FATTORIE DIDATTICHE AD AFRAGOLA CHE SONO APERTE DOMANI ALLE 8 DI PROVINCIA '}

#### Inizio preparazione dati per training modello di classificazione

In [205]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [206]:
cv.fit(df.sentences)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [66]:
print(f"Il vocabolario contiene {len(cv.vocabulary_)} parole")

cv.vocabulary_

Il vocabolario contiene 18 parole


{'ci': 3,
 'sono': 17,
 'fattorie': 9,
 'didattiche': 8,
 'in': 10,
 'provincia': 14,
 'di': 6,
 'con': 5,
 'produzione': 13,
 'categoria': 1,
 'quante': 15,
 'sede': 16,
 'nella': 11,
 'che': 2,
 'producono': 12,
 'didattche': 7,
 'ad': 0,
 'coltivazione': 4}

In [68]:
fake_list = list()

for i in range(300):
    fake_list.append(" ".join(random.choices(list(cv.vocabulary_.keys()), k = 10)))

In [69]:
fake_list[:10]

['ad ad con didattche coltivazione didattche ci producono nella che',
 'in di produzione didattiche categoria sono provincia sono produzione con',
 'con fattorie coltivazione di nella ci ad quante con che',
 'didattche fattorie didattche categoria sono nella ad di didattche categoria',
 'sono coltivazione nella didattche didattche categoria nella provincia quante con',
 'didattche coltivazione con produzione in sede categoria categoria didattche quante',
 'didattche provincia nella in ad sede produzione categoria in provincia',
 'ad didattche categoria ci ad sono ci ci di ad',
 'ci categoria produzione che provincia con in sono sede sede',
 'nella nella con producono in con didattche didattiche quante coltivazione']

In [70]:
df_fake = pd.DataFrame({"sentences":fake_list, "intents":"fake"})
df_fake.head()

,sentences,intents
0,ad ad con didattche coltivazione didattche ci ...,fake
1,in di produzione didattiche categoria sono pro...,fake
2,con fattorie coltivazione di nella ci ad quant...,fake
3,didattche fattorie didattche categoria sono ne...,fake
4,sono coltivazione nella didattche didattche ca...,fake


In [71]:
df = pd.concat([df, df_fake], axis=0, ignore_index=True)
df.tail()

,sentences,intents
1295,fattorie di didattche didattche fattorie didat...,fake
1296,ad quante coltivazione nella provincia produco...,fake
1297,didattiche quante in sede ad nella ad in categ...,fake
1298,produzione producono ci producono quante didat...,fake
1299,nella didattiche didattiche quante sede produc...,fake


In [72]:
import sklearn

#### Preparazione del target per il training

In [73]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [74]:
le.fit(df.intents.unique())

for n, cls in enumerate(le.classes_):
    print(f"{cls} --> {n}")

fake --> 0
user_AskCity --> 1
user_AskProv --> 2
user_AskProvWithParams --> 3


In [75]:
labels_categories = le.transform(df.intents.values)

In [76]:
le.inverse_transform([1])[0]

'user_AskCity'

#### Trasformazione e preprocessing della frase per il training del modello

In [77]:
print(df.sentences[0])

np.max(cv.transform(df.sentences[0].split()).toarray(), axis=0)

CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI PROVINCIA CON PRODUZIONE DI CATEGORIA 


array([0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1])

In [78]:
print(f"""
Il dataset si compone di {cv.transform(df['sentences'].values).toarray().shape[0]} frasi
con n.{cv.transform(df['sentences'].values).toarray().shape[1]} colonne 
(infatti il vocabolario contiene 18 parole)
""")


Il dataset si compone di 1300 frasi
con n.18 colonne 
(infatti il vocabolario contiene 18 parole)



In [79]:
from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier(fit_intercept=False, loss='log')

In [80]:
classifier.fit(X=cv.transform(df['sentences'].values).toarray(), y=labels_categories)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=False,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

#### Esempio su una frase contenuta nel training set

In [81]:
df['intents'].values[0]

'user_AskProvWithParams'

In [82]:
df['sentences'].values[0]

'CI SONO FATTORIE DIDATTICHE IN PROVINCIA DI PROVINCIA CON PRODUZIONE DI CATEGORIA '

In [83]:
y_pred = classifier.predict([np.max(cv.transform(df['sentences'].values[0].split()).toarray(), axis = 0)])

In [84]:
le.inverse_transform(y_pred)[0]

'user_AskProvWithParams'

In [85]:
from sklearn.metrics import confusion_matrix

In [86]:
predicted_categories = [classifier.predict([np.max(cv.transform(sentence.split()).toarray(), axis = 0)])[0] for sentence in df.sentences.values]

In [87]:
cm = confusion_matrix(labels_categories, predicted_categories)
cm

array([[290,   2,   0,   8],
       [  0, 210,   0,   0],
       [  0,   0, 332,   0],
       [  0,   0,   0, 458]])

In [88]:
print(f"Accuracy: {sum(cm.diagonal()) / cm.sum() * 100}%")

Accuracy: 99.23076923076923%


#### Test classificazione su nuova frase

In [89]:
processed_sentence = add_slots(nuova_frase, crf)['replaced_sentence']
processed_sentence

'CI SONO FATTORIE DIDATTICHE AD SEDE'

In [90]:
y_pred = classifier.predict([np.max(cv.transform(processed_sentence.split()).toarray(), axis = 0)])[0]
le.inverse_transform([y_pred])[0]

'user_AskCity'

##### Trasformiamo tutto in una funzione

In [91]:
def get_intents_and_slots(sentence:str, model:sklearn_crfsuite.estimator.CRF, cv:sklearn.feature_extraction.text.CountVectorizer, le:sklearn.preprocessing._label.LabelEncoder, threasold = 0.25) -> dict:
    
    """
    Estrae gli intents e gli slots dalla frase
    """
    
    sentence_dict = add_slots(sentence, crf)
    
    sentence = sentence_dict['replaced_sentence']
    
    arr = cv.transform(add_slots(sentence, crf)['replaced_sentence'].split()).toarray()
    
    arr = np.max(arr, axis = 0)
    
    probs = model.predict_proba([arr])[0]
            
    df = pd.DataFrame({"classes":le.classes_, "probs":probs})
    df.sort_values("probs", ascending=False, inplace=True)    
    classes_with_prob = df[df['probs']>threasold].to_dict('records')
    sentence_dict['intents'] = classes_with_prob
    
    max_intent = le.classes_[np.argmax(probs)]
    sentence_dict['max_intent'] = max_intent

    
    return sentence_dict
    

In [92]:
data = get_intents_and_slots(nuova_frase, classifier, cv, le)

In [95]:
pprint.pprint(data)

{'intents': [{'classes': 'user_AskCity', 'probs': 0.9996863126884761}],
 'max_intent': 'user_AskCity',
 'replaced_sentence': 'CI SONO FATTORIE DIDATTICHE AD SEDE',
 'sentence': 'CI SONO FATTORIE DIDATTICHE AD AFRAGOLA',
 'slots': defaultdict(None, {'SEDE': 'AFRAGOLA'})}


## Parte 4

### Combinare le info estratte dalla frase per filtrare un semplice dataset OpenData

Comprensione del dialogo e della costruzione della risposta

In [235]:
dialogs = {v[0]:v[1] for v in list(dialogs.values())}

pprint.pprint(dialogs)

{'user_AskCity': 'bot_ReplyCity ',
 'user_AskProv': 'bot_ReplyProv',
 'user_AskProvWithParams': 'bot_ReplyProvWithParams'}


In [236]:
bot_sentences

{'bot_ReplyProv': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche'],
 'bot_ReplyCity ': ['Nella città di [SEDE] ci sono %d fattorie didattiche'],
 'bot_ReplyProvWithParams': ['Nella provincia di [PROVINCIA] ci sono %d fattorie didattiche con le caratteristiche richieste']}

In [259]:
"""
Estrazione intent
"""

if data['intents'][0]['probs'] > 0.75:

    intent = data['max_intent']
    
else:
    
    intent = 'fake'

intent

'fake'

In [260]:
# data['intents'][0]['probs'] > 0.75

In [261]:
"""
Estrazione risposta
"""

try:

    reply = dialogs[intent]

except:
    
    reply = "fake"
    
reply

'fake'

In [266]:
"""
Estrazione frase con parametri
"""

if reply != "fake":

    reply_str = random.choice(bot_sentences[reply])

reply_str

'Nella provincia di [PROVINdf ci sono %d fattorie didattiche con le caratteristiche richieste'

Estrazione dei dati per la sostituzione dei parametri

In [267]:
query_list = list()

for k,v in data['slots'].items():
    query_list.append(f"{k} == '{v}'")
    
    k = list(data['slots'].keys())[0]
    k = f"[{k}]"
    
    v = list(data['slots'].values())[0]
    
    reply_str = re.sub(f"[{k}+]", v, reply_str)
        
query = " and ".join(query_list)
print(query)

CATEGORIA == 'df'


In [268]:
reply_str

'Nella provincia di [PROVINdf ci sono %d fattorie didattiche con le caratteristiche richieste'

In [269]:
df_query = pd.read_csv('db_esempio.csv')
df_query.head()

,NOME,SEDE,PROVINCIA,CATEGORIA
0,Azienda Sperimentale Regionale Improsta,EBOLI,SALERNO,ALLEVAMENTO
1,"Museo della Civiltà Contadina ""Michele Russo""",SOMMA VESUVIANA,NAPOLI,ALLEVAMENTO
2,Masseria Panico di Eredi Beneduce Ettore,SOMMA VESUVIANA,NAPOLI,ALLEVAMENTO
3,Cooperativa Agrituristica La Ginestra,VICO EQUENSE,NAPOLI,ALLEVAMENTO
4,Agriturismo Costiera Amalfitana,TRAMONTI,SALERNO,ALLEVAMENTO


In [270]:
n = df_query.query(query)['NOME'].drop_duplicates().count()

print(reply_str % (n))

Nella provincia di [PROVINdf ci sono 0 fattorie didattiche con le caratteristiche richieste


In [96]:
# if data['max_intent'] == 'user_AskProvWithParams':
    
#     n = df_query.query(query)['NOME'].drop_duplicates().count()
    
#     print(reply_str % (n))
    
# elif data['max_intent'] == 'user_AskProv':
    
#     n = df_query.query(query)['NOME'].drop_duplicates().count()
    
#     print(reply_str % (n))

# elif data['max_intent'] == 'user_AskCity':
    
#     n = df_query.query(query)['NOME'].drop_duplicates().count()
    
#     print(reply_str % (n))

# else:
#     print("Non ho capito")

##### Adesso raggruppiamo tutto in una funzione

In [296]:
def bot_reply(sentence:str, model:sklearn_crfsuite.estimator.CRF, cv:sklearn.feature_extraction.text.CountVectorizer, le:sklearn.preprocessing._label.LabelEncoder, dialogs:dict, bot_sentences:dict, db:str = 'db_esempio.csv', threasold = 0.25) -> dict:
    
    """
    Genera la risposta
    """
    
    sentence = sentence.upper()
    
    data = get_intents_and_slots(sentence, model, cv, le, threasold)
    
#     intent = data['max_intent']

    pprint.pprint(data)
    print("---------------------------------------------")
    
    if data['intents'][0]['probs'] > 0.75 and not 'fake':
        
        intent = data['max_intent']
        
        reply = dialogs[intent]

        reply_str = random.choice(bot_sentences[reply])
       
        query_list = list()

        for k,v in data['slots'].items():
            query_list.append(f"{k} == '{v}'")

            k = list(data['slots'].keys())[0]
            k = f"[{k}]"

            print(k)

            v = list(data['slots'].values())[0]

            reply_str = re.sub(fr"[{k}+]", v, reply_str)

        query = " and ".join(query_list)

        df_query = pd.read_csv(db)

        n = df_query.query(query)['NOME'].drop_duplicates().count()
        
        return reply_str % (n)
    
    else:

        intent = 'Non ho capito, riformula meglio la tua domanda'

        return intent

In [297]:
nuova_frase = "QUANTE FATTORIE DIDATTICHE CI SONO AD ACCIAROLI ?"

bot_reply(nuova_frase, classifier, cv, le, dialogs, bot_sentences)

{'intents': [{'classes': 'user_AskCity', 'probs': 0.9592778810017446}],
 'max_intent': 'user_AskCity',
 'replaced_sentence': 'QUANTE FATTORIE DIDATTICHE CI SONO AD SEDE ',
 'sentence': 'QUANTE FATTORIE DIDATTICHE CI SONO AD ACCIAROLI ',
 'slots': defaultdict(None, {'SEDE': 'ACCIAROLI'})}
---------------------------------------------


'Non ho capito, riformula meglio la tua domanda'

In [294]:
while True:
    
    try:
    
        nuova_frase = input("Chiedi qualcosa:")

        print(bot_reply(nuova_frase, classifier, cv, le, dialogs, bot_sentences))
        
    except Exception as KeyboardInterrupt:
        break

Chiedi qualcosa:Il primo aereo per Milano ?
{'intents': [{'classes': 'user_AskProvWithParams', 'probs': 0.5732029556565345},
             {'classes': 'fake', 'probs': 0.4136310947671571}],
 'max_intent': 'user_AskProvWithParams',
 'replaced_sentence': 'IL PRIMO AEREO PER CATEGORIA ',
 'sentence': 'IL PRIMO AEREO PER MILANO ',
 'slots': defaultdict(None, {'CATEGORIA': 'MILANO'})}
Non ho capito, riformula meglio la tua domanda


KeyboardInterrupt: Interrupted by user